In [1]:
import numpy as np
from random import random

from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import QControlBox
from pytket.circuit import CircBox


In [61]:
# m is precision of answer
def iter_circ(m_bits, eigenstate: Circuit, unitary: Circuit) -> Circuit:  
    # create base of circuit to return
    iter_circ: Circuit = Circuit(1, m_bits + 1)
    qreg = iter_circ.get_q_register("q")
    creg = iter_circ.get_c_register("c")
    ancil = qreg[0]
    n_state_prep_qubits = eigenstate.n_qubits
    sreg = iter_circ.add_q_register("psi", n_state_prep_qubits)
    iter_circ.add_circuit(eigenstate, list(sreg))

    # Create a controlled unitary with a single control qubit
    unitary.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary), 1)
    
    for j in range(m_bits, 0, -1):
        # add hadamard to ancillary qubit
        iter_circ.H(0)
        # add unitaries
        for _ in range(2 ** (j - 1)):
            iter_circ.add_qcontrolbox(controlled_u_gate, [ancil] + list(sreg))

        # add rotations
        for expn in range(m_bits + 1 - j, 1, -1):
            phase = - 2 * np.pi / (2 ** expn)
            # this is the problematic line, I believe
            iter_circ.U1(phase, 0, condition = creg[expn + j - 1])
            print(expn, expn + j - 1)
        
        iter_circ.H(0)
        iter_circ.Measure(ancil, creg[j])
        iter_circ.Reset(0)
        iter_circ.X(0)

    # dummy just in case
    iter_circ.Measure(ancil, creg[0])
    # Add the initial state
    return iter_circ
    # # Create the unitary:
    # unitary.name = "U"
    # cu_gate = QControlBox(CircBox(unitary), 1)
    # render_circuit_jupyter(iter_circ)
    # pass

## Call the function, run experiments and stuff

In [62]:
#input_angle = 0.5 
# From 0 to 1
input_angle = 0.73
initial_state = Circuit(1).X(0) #Make sure to just add quantum channels here
unitary = Circuit(1).U1(input_angle, 0)
circuit_iter_to_render = iter_circ(2, 
    eigenstate=initial_state, unitary=unitary
)

render_circuit_jupyter(circuit_iter_to_render)

2 2


In [11]:
import pytket.backends

In [4]:
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from datetime import datetime

In [56]:
phase_est_project = Nexus().new_project(f"IPE_test8 - {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1LE", user_group="iQuHACK_2024")
backend = NexusBackend(
    backend_config= configuration, 
    project= phase_est_project
)


New project created: IPE_test8 - 2024-02-04 07:32:51.127823

Started using project with name: IPE_test8 - 2024-02-04 07:32:51.127823


In [57]:
# PATCH 
# decompose any Boxes in the circuit (acts in place on the circuit object)
from pytket.passes import DecomposeBoxes

DecomposeBoxes().apply(circuit_iter_to_render)

True

In [58]:
compiled_circ = backend.get_compiled_circuit(circuit_iter_to_render)

RuntimeError: compile-job-adnexed-dusky-manliness-procera: Circuit has encountered an error during compilation.
CompileJobItem 0: <class 'grpclib.exceptions.GRPCError'>: (<Status.INTERNAL: 13>, 'unknown variant `Phase`, expected one of `Input`, `Output`, `Create`, `Discard`, `ClInput`, `ClOutput`, `Barrier`, `Label`, `Branch`, `Goto`, `Stop`, `ClassicalTransform`, `SetBits`, `CopyBits`, `RangePredicate`, `ExplicitPredicate`, `ExplicitModifier`, `MultiBit`, `Z`, `X`, `Y`, `S`, `Sdg`, `T`, `Tdg`, `V`, `Vdg`, `SX`, `SXdg`, `H`, `Rx`, `Ry`, `Rz`, `U3`, `U2`, `U1`, `TK1`, `TK2`, `CX`, `CY`, `CZ`, `CH`, `CV`, `CVdg`, `CSX`, `CSXdg`, `CRz`, `CRx`, `CRy`, `CU1`, `CU3`, `PhaseGadget`, `CCX`, `SWAP`, `CSWAP`, `BRIDGE`, `noop`, `Measure`, `Collapse`, `Reset`, `ECR`, `ISWAP`, `PhasedX`, `NPhasedX`, `ZZMax`, `XXPhase`, `YYPhase`, `ZZPhase`, `XXPhase3`, `ESWAP`, `FSim`, `Sycamore`, `ISWAPMax`, `PhasedISWAP`, `CnRy`, `CnX`, `CircBox`, `Unitary1qBox`, `Unitary2qBox`, `Unitary3qBox`, `ExpBox`, `PauliEx